# Lab 5.1 Fine-Tuning

In this notebook you will explore fine-tuning a CNN to classify pet breeds.

In [9]:
device = 'cpu'
#device = 'cuda' # set this if using GPU on Colab for example

Here is some code to download a prepared version of the [Oxford-IIIT Pet dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/).

In [10]:
import os
if not os.path.exists('oxford_pets.zip'):
  !wget "https://www.dropbox.com/scl/fi/p49ifha27c2u3uptfj42w/oxford_pets_corrected.zip?rlkey=dwk3dsptzir8v846imsq6bgw3&dl=1" -O oxford_pets.zip
  !unzip -qq oxford_pets.zip

In [11]:
import torch
import torch.nn as nn
import torchvision
import torchmetrics
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

The following code sets up dataset and dataloaders for the data.  It creates two versions: the `no_transform` version does not apply any transformations, and the other one applies the appropriate transformations for the VGG16 model (resize, recropping, centering, and scaling).

In [ ]:
transforms = torchvision.models.VGG16_Weights.IMAGENET1K_V1.transforms()

train_ds_no_transform = torchvision.datasets.ImageFolder(
    'oxford_pets/train',
)
train_ds = torchvision.datasets.ImageFolder(
    'oxford_pets/train',
    transform=transforms
)
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)
test_ds_no_transform = torchvision.datasets.ImageFolder(
    'oxford_pets/test',
)
test_ds = torchvision.datasets.ImageFolder(
    'oxford_pets/test',
    transform=transforms
)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=32, shuffle=False)

In [ ]:
train_ds

In [ ]:
image,label = next(iter(train_ds_no_transform))
plt.imshow(image)
plt.title(train_ds.classes[label])

Now we will grab the VGG16 model with pretrained weights.

In [ ]:
vgg16 = torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.IMAGENET1K_V1).to(device)

In [ ]:
vgg16

### Exercises

1. Iterate through the parameters of the `features` part of VGG16 and set `requires_grad=False` on each paramter.

2. Replace the `classifier` part of the network with a new `Sequential` model that has 35 outputs for the 35 pet breeds.

*Note:* You can do `vgg16.classifier = nn.Sequential(...).to(device)`.

3. Use the following code to check the test set accuracy of the model *before* training it.  Interpret the result.

In [ ]:
metric = torchmetrics.Accuracy('multiclass',num_classes=35).to(device)
metric.reset()
for x_batch, y_batch in test_loader:
    y_pred = vgg16(x_batch.to(device))
    metric(y_pred, y_batch.to(device))
metric.compute()

5. Now train the model on the training set for three epochs.

6. Evaluate the accuracy of the fine-tuned model on the test set.

7. The following code will show some test images with the correct and predicted labels.

In [ ]:
vgg16.eval()
preds = []
for x_batch, y_batch in test_loader:
    y_pred = vgg16(x_batch.to(device))
    y_pred = torch.argmax(y_pred,dim=-1)
    preds.append(y_pred.detach().cpu().numpy())
preds = np.concatenate(preds,axis=0)

In [ ]:
i = 0
for (im,label),pred in zip(test_ds_no_transform,preds):
  if i%20 == 0:
    plt.imshow(im)
    plt.title(f'correct: {test_ds.classes[label]} | predicted: {test_ds.classes[pred]}')
    plt.show()
  i += 1